# Examples

## Query Example

This notebook will cover example usage of `Result` record. As a note we will be using the MolSSI QCArchive server as a data source. Any `id`s used in this example will not be valid for local servers.

In [2]:
import qcfractal.interface as ptl
client = ptl.FractalClient()
client

FractalClient(server_name='The MolSSI QCArchive Server', address='https://api.qcarchive.molssi.org:443/', username='None')

We can query results from the database based off a variety of values, but for this example we will query a known result from the database.

In [20]:
record = client.query_results(id=1615129)[0]
record

<ResultRecord(id='1615129' status='COMPLETE')>

There are a variety of helped functions on this

In [21]:
record.get_molecule()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<Molecule(name='C8H11N3O' formula='C8H11N3O' hash='f0e2dbc')>

In [37]:
print(f"Program: {record.program} Model: {record.method.upper()}/{record.basis} ({record.driver})")

Program: psi4 Model: B3LYP-D3(BJ)/dzvp (gradient)


Since the primary `driver` of this computation a `gradient` is in the `record.return_result` slot as a NumPy array. In addition, all parsed intermediates quantities can be found in the `record.properties` slot for intermediate data.

In [38]:
record.return_result[:3]

array([[-1.0183802691625713e-05,  2.4983023230544212e-05,
         4.2626711726293591e-05],
       [ 4.8518698310640986e-05,  2.8649537718356986e-06,
        -2.5993909127035813e-05],
       [-1.4760360682320329e-05, -2.1228208508431367e-05,
        -4.6180180609961721e-05]])

In [28]:
record.properties.dict()

{'calcinfo_nbasis': 190,
 'calcinfo_nmo': 190,
 'calcinfo_nalpha': 44,
 'calcinfo_nbeta': 44,
 'calcinfo_natom': 23,
 'nuclear_repulsion_energy': 693.2680312650867,
 'return_energy': -551.0714449809266,
 'scf_one_electron_energy': -2121.681087605063,
 'scf_two_electron_energy': 939.395572266993,
 'scf_xc_energy': -62.01215606794322,
 'scf_dispersion_correction_energy': -0.04180484,
 'scf_dipole_moment': [-0.43157518923938226,
  3.5732964493427315,
  -0.35820710190059735],
 'scf_total_energy': -551.0714449809266,
 'scf_iterations': 14}

Additional data such as the output of the quantum chemistry program can be queried as well:

In [34]:
print(record.get_stdout()[:245])


  Memory set to  60.800 GiB by Python driver.
gradient() will perform analytic gradient computation.

*** tstart() called on ca167
*** at Fri May 17 00:25:45 2019

   => Loading Basis Set <=

    Name: DZVP
    Role: ORBITAL
    Keyword: BASIS



## Compute Example

Computation of a result can be accomplished by specifying all parameters to a quantum chemistry computation and a molecule. An example can be seen below using a FractalSnowflake instance.

In [39]:
from qcfractal import FractalSnowflakeHandler
snowflake = FractalSnowflakeHandler()
client = snowflake.client()
client

FractalClient(server_name='db_0fa4f6ef_db7e_4f2e_9b7e_7c31cd9790d9', address='https://localhost:55881/', username='None')

In [45]:
methane = ptl.Molecule.from_data("pubchem:methane")
methane

	Searching PubChem database for methane (single best match returned)
	Found 1 result(s)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<Molecule(name='IUPAC methane' formula='CH4' hash='b4057dd')>

To run a quantum chemistry computation on this methane molecule we need to specify the full input as shown below. It should be noted that this function is also organized in such a way where the computation of many molecules with the same level of theory is most efficient.

In [54]:
#                          program, method, basis,   driver,  keywords, molecule
compute = client.add_compute('psi4', 'hf', 'sto-3g', 'energy', None, [methane])
compute

<ComputeResponse(nsubmitted=0 nexisting=1)>

The `id`s of the submitted compute can then be queried and examined. As a note the computation is not instantaneous, you may need to wait a moment and requery for this small molecule.

In [52]:
result = client.query_results(id=compute.ids)[0]
result

<ResultRecord(id='1' status='COMPLETE')>

In [51]:
result.return_result

-39.726728729639106